<a href="https://colab.research.google.com/github/Anririka/demo/blob/main/taitanic_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google driveからマウントする

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# test_filepath = '/content/drive/MyDrive/1cGjIvv4O6Wujkdd8_ySmTxgAEIwciCU/test.csv'
# train_filepath = '/content/drive/MyDrive/1cGjIvv4O6Wujkdd8_ySmTxgAEIwciCU/train.csv'

In [8]:
test_filepath = '/content/drive/MyDrive/programing/demo_taitanic/data/test.csv'
train_filepath = '/content/drive/MyDrive/programing/demo_taitanic/data/train.csv'

In [9]:
import pandas as pd

test_df = pd.read_csv(test_filepath)
train_df = pd.read_csv(train_filepath)

In [10]:
import os

if os.path.exists(test_filepath) and os.path.exists(train_filepath):
  print("ファイルが存在します。")
else:
  print("ファイルが存在しません。パスを確認してください。")

ファイルが存在します。


In [12]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#　Kー近傍法

In [13]:
train_df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [14]:
train_df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [15]:
test_df.dtypes

,0
PassengerId,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64
Cabin,object


In [16]:
test_df.isnull().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


In [20]:
x = train_df[['Pclass','SibSp','Parch']]
y = train_df[['Survived']]

In [23]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [25]:
y

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [26]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [27]:
knn.score(x_test, y_test)

0.659217877094972

In [28]:
x_for_submit= test_df[['Pclass','SibSp','Parch']]
submit = test_df[['PassengerId']]
submit['Survived'] = knn.predict(x_for_submit)
submit

<ipython-input-28-84427a8a4574>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['Survived'] = knn.predict(x_for_submit)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [31]:
from operator import index
submit.to_csv('/content/drive/MyDrive/programing/demo_taitanic/submit01.csv', index=False)

# 数値以外も特徴量に追加する

In [32]:
x = train_df[['Pclass','SibSp','Parch','Sex']]
y = train_df[['Survived']]

In [39]:
x = pd.get_dummies(x, columns=['Sex','Pclass'])

In [40]:
x

,SibSp,Parch,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,1,0,False,True,False,False,True
1,1,0,True,False,True,False,False
2,0,0,True,False,False,False,True
3,1,0,True,False,True,False,False
4,0,0,False,True,False,False,True
...,...,...,...,...,...,...,...
886,0,0,False,True,False,True,False
887,0,0,True,False,True,False,False
888,1,2,True,False,False,False,True
889,0,0,False,True,True,False,False


In [41]:
columns_to_convert = ['Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3']
x[columns_to_convert] = x[columns_to_convert].astype(int)

In [42]:
x

,SibSp,Parch,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,1,0,0,1,0,0,1
1,1,0,1,0,1,0,0
2,0,0,1,0,0,0,1
3,1,0,1,0,1,0,0
4,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...
886,0,0,0,1,0,1,0
887,0,0,1,0,1,0,0
888,1,2,1,0,0,0,1
889,0,0,0,1,1,0,0


In [43]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

knn.score(x_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.7821229050279329

# nullを含むfareを追加する

In [48]:
x = train_df[['Pclass','SibSp','Parch','Sex','Fare']]
y = train_df[['Survived']]
x = pd.get_dummies(x, columns=['Sex','Pclass'])
columns_to_convert = ['Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3']
x[columns_to_convert] = x[columns_to_convert].astype(int)

In [49]:
x

,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,1,0,7.2500,0,1,0,0,1
1,1,0,71.2833,1,0,1,0,0
2,0,0,7.9250,1,0,0,0,1
3,1,0,53.1000,1,0,1,0,0
4,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...
886,0,0,13.0000,0,1,0,1,0
887,0,0,30.0000,1,0,1,0,0
888,1,2,23.4500,1,0,0,0,1
889,0,0,30.0000,0,1,1,0,0


In [50]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

knn.score(x_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.7932960893854749

In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

knn = KNeighborsClassifier()
knn.fit(x_train_scaled, y_train)

knn.score(x_test_scaled, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.8044692737430168

In [53]:
x_for_submit= test_df[['Pclass','SibSp','Parch','Sex','Fare']]
submit = test_df[['PassengerId']]

x_for_submit = pd.get_dummies(x_for_submit, columns=['Sex','Pclass'])
columns_to_convert = ['Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3']
x_for_submit[columns_to_convert] = x_for_submit[columns_to_convert].astype(int)

x_for_submit['Fare'] =x_for_submit['Fare'].fillna(x_for_submit['Fare'].mean())

scaler = StandardScaler()
scaler.fit(x_for_submit)

x_for_submit_scaled = scaler.transform(x_for_submit)

submit['Survived'] = knn.predict(x_for_submit_scaled)
submit

<ipython-input-53-b95770348752>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['Survived'] = knn.predict(x_for_submit_scaled)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [54]:
submit.to_csv('/content/drive/MyDrive/programing/demo_taitanic/submit02.csv', index=False)

In [59]:
x = train_df[['Pclass','SibSp','Parch','Sex','Fare']]
y = train_df[['Survived']]

x = pd.get_dummies(x, columns=['Sex','Pclass'])
columns_to_convert = ['Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3']
x[columns_to_convert] = x[columns_to_convert].astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)


scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)


In [60]:
from sklearn.svm import LinearSVC

svc= LinearSVC()
svc.fit(x_train_scaled, y_train)

svc.score(x_test_scaled, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7892376681614349

In [61]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_train_scaled, y_train)

rf.score(x_test_scaled, y_test)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.7892376681614349

In [62]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(x_train_scaled, y_train)

gb.score(x_test_scaled, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7982062780269058

In [63]:
x_for_submit= test_df[['Pclass','SibSp','Parch','Sex','Fare']]
submit = test_df[['PassengerId']]

x_for_submit = pd.get_dummies(x_for_submit, columns=['Sex','Pclass'])
columns_to_convert = ['Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3']
x_for_submit[columns_to_convert] = x_for_submit[columns_to_convert].astype(int)

x_for_submit['Fare'] =x_for_submit['Fare'].fillna(x_for_submit['Fare'].mean())

x_for_submit_scaled = scaler.transform(x_for_submit)

submit['Survived'] = gb.predict(x_for_submit_scaled)
submit

<ipython-input-63-6b8f1a9e0b43>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['Survived'] = gb.predict(x_for_submit_scaled)


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [64]:
submit.to_csv('/content/drive/MyDrive/programing/demo_taitanic/submit03.csv', index=False)